In [ ]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def abrir_formulario_inicial(timeout: int = 15) -> webdriver.Chrome:
    """
    1) Arranca Chrome en modo headless (sin ventana) pero con un User-Agent real
       para que el WAF no te bloquee.
    2) Navega a la URL de inicio de la Registraduría.
    3) Hace clic en el botón “Ingresar usuario público” (id="controlador:consultasId").
    4) Espera a que aparezca el <select id="searchForm:tiposBusqueda">, para confirmar
       que ya estamos en la sección del formulario.
    5) Devuelve el webdriver listo en la página del formulario.

    Si en algún paso falla (timeout, no encuentra el botón o el select),
    lanza RuntimeError con un mensaje descriptivo.
    """

    # --------------- 1) Configurar ChromeOptions ---------------
    chrome_opts = Options()
    # Usamos headless para que no abra ventana, pero forzamos un User-Agent “normal”
    chrome_opts.add_argument("--headless=new")
    chrome_opts.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    )
    # Ignorar posibles errores de certificado y sandbox
    chrome_opts.add_argument("--ignore-certificate-errors")
    chrome_opts.add_argument("--allow-insecure-localhost")
    chrome_opts.add_argument("--no-sandbox")
    chrome_opts.add_argument("--disable-dev-shm-usage")
    chrome_opts.add_argument("--window-size=1920,1080")
    chrome_opts.add_argument("--log-level=3")  # silencio de logs

    driver = webdriver.Chrome(options=chrome_opts)

    try:
        # --------------- 2) Cargar la página de inicio ---------------
        url_inicio = "https://consultasrc.registraduria.gov.co:28080/ProyectoSCCRC/"
        driver.get(url_inicio)

        # Opcional: verificación rápida del título para asegurarnos de que cargó bien
        print("▶ Título de la página inicial:", driver.title)

        # --------------- 3) Hacer clic en “Ingresar usuario público” ---------------
        # Esperamos hasta 10 segundos a que el botón sea clickeable
        boton = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "controlador:consultasId"))
        )
        boton.click()

        # --------------- 4) Esperar a que aparezca el <select> del formulario ---------------
        # El dropdown que queremos encontrar es: <select id="searchForm:tiposBusqueda" …>
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.ID, "searchForm:tiposBusqueda"))
        )

        print("✔️ Llegaste al formulario correctamente.")
        return driver

    except Exception as e:
        driver.quit()
        raise RuntimeError(f"Error en abrir_formulario_inicial: {e}")






In [ ]:
import os
import time
import io
import base64
import random
import requests
import cv2
import numpy as np
from PIL import Image
import pytesseract
import easyocr

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# ------------------------------------------------------------
# CONFIGURACIÓN GENERAL
# ------------------------------------------------------------

# 1) Ruta a tesseract.exe (asegúrate de que esté instalada y en esa ruta)
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\crist\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

# 2) Inicializamos un reader de EasyOCR
reader_eo = easyocr.Reader(['en'], gpu=False)

# 3) Carpeta donde guardaremos todos los CAPTCHAs y logs
CARPETA_CAPTCHAS = r"C:\Users\crist\Downloads\captcha"
os.makedirs(CARPETA_CAPTCHAS, exist_ok=True)

# 4) Nombre del archivo de log dentro de la carpeta de CAPTCHAs
RUTA_LOG = os.path.join(CARPETA_CAPTCHAS, "log_captcha.txt")


# ------------------------------------------------------------
# 1) Funciones de preprocesamiento “variacional” para EasyOCR
# ------------------------------------------------------------

def preproc_variations(pil_img: Image.Image):
    """
    Genera 20 variantes diferentes de preprocesamiento de la misma imagen.
    - La variante 1 está fija con block_size=17, C=3, kernel_size=4.
    - Las siguientes 19 variantes se eligen aleatoriamente.
    Devuelve lista de tuplas [(imagen_procesada, config), ...].
    """
    img_cv = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    gray_original = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    variantes = []

    # Variante fija: block=17, C=3, kernel=4
    block_size = 17
    C = 3
    k = 4
    thresh = cv2.adaptiveThreshold(
        gray_original, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        block_size, C
    )
    kernel = np.ones((k, k), np.uint8)
    proc = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    proc = cv2.morphologyEx(proc, cv2.MORPH_OPEN, kernel, iterations=1)
    proc_final = cv2.bitwise_not(proc)
    variantes.append((Image.fromarray(proc_final), {'block_size': block_size, 'C': C, 'kernel_size': k}))

    # 19 variantes aleatorias
    elecciones_block = [11, 13, 15, 17, 19, 21]
    for i in range(2, 21):
        block_size = random.choice(elecciones_block)
        C = random.randint(1, 5)
        thresh = cv2.adaptiveThreshold(
            gray_original, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            block_size, C
        )
        k = random.choice([2, 3, 4])
        kernel = np.ones((k, k), np.uint8)

        if k >= 3:
            proc = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
            proc = cv2.morphologyEx(proc, cv2.MORPH_OPEN, kernel, iterations=1)
        else:
            proc = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

        proc_final = cv2.bitwise_not(proc)
        variantes.append((Image.fromarray(proc_final), {'block_size': block_size, 'C': C, 'kernel_size': k}))

    return variantes


# ------------------------------------------------------------
# 2) Función que aplica EasyOCR a una imagen precargada
# ------------------------------------------------------------

def ocr_easy_single(pil_img: Image.Image) -> str:
    """
    Llama a EasyOCR y devuelve la primera cadena de 5 caracteres alfanuméricos.
    Si no existe, devuelve "".
    """
    arr = np.array(pil_img.convert('RGB'))
    results = reader_eo.readtext(arr)

    for (_bbox, texto, _conf) in results:
        txt = texto.strip().replace(" ", "")
        txt = ''.join(c for c in txt if c.isalnum())
        if len(txt) == 5:
            return txt

    return ""


# ------------------------------------------------------------
# 3) Función que, dada la imagen original, intenta resolver con EasyOCR
# ------------------------------------------------------------

def resolver_captcha_con_easy_ensembles(pil_img: Image.Image, cedula: str, intento: int) -> (str, bool):
    """
    1) Genera 20 variantes, la primera fija y 19 aleatorias.
    2) Aplica EasyOCR a cada una.
    3) Si alguna cadena de 5 caracteres aparece >= 5 veces, devuelve (texto, True).
    4) Si no, devuelve ("", False).
    Registra cada pasada y configuración en el log.
    """
    variantes = preproc_variations(pil_img)
    resultados = []

    for idx, (var_img, cfg) in enumerate(variantes, start=1):
        texto = ocr_easy_single(var_img)
        resultados.append((idx, cfg, texto))

    # Registrar en log
    with open(RUTA_LOG, "a", encoding="utf-8") as logf:
        logf.write(f"CÉDULA: {cedula} | EASYOCR_INTENTO: {intento}\n")
        for (idx, cfg, texto) in resultados:
            logf.write(
                f"  Variante {idx:2d} | block={cfg['block_size']:2d} "
                f"C={cfg['C']:1d} kernel={cfg['kernel_size']:1d} | OCR='{texto}'\n"
            )
        logf.write("\n")

    # Contar apariciones
    contador = {}
    for (_idx, _cfg, txt) in resultados:
        if len(txt) == 5:
            contador[txt] = contador.get(txt, 0) + 1

    for txt, count in contador.items():
        if count >= 5:
            return (txt, True)

    return ("", False)


# ------------------------------------------------------------
# 4) Función para “limpiar la línea más larga” de la imagen
# ------------------------------------------------------------

def remove_longest_line_random(pil_img: Image.Image, cedula: str, intento: int, iter_ed: int) -> (Image.Image, dict):
    """
    Detecta la línea más larga mediante HoughLinesP y la borra con
    grosor aleatorio. Devuelve (imagen_editada, config).
    Config contiene los parámetros utilizados (canny thresholds, minLineLength, maxLineGap, thickness).
    Si no se encuentra línea, retorna la imagen original y config={'found_line': False}.
    """
    img_cv = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    # Parámetros aleatorios para Canny y HoughLinesP
    canny_thresh1 = random.randint(50, 100)
    canny_thresh2 = random.randint(150, 200)
    min_line_len = random.randint(30, 60)
    max_line_gap = random.randint(5, 20)

    edges = cv2.Canny(gray, canny_thresh1, canny_thresh2)
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=30,
                            minLineLength=min_line_len, maxLineGap=max_line_gap)

    config = {
        'canny_thresh1': canny_thresh1,
        'canny_thresh2': canny_thresh2,
        'min_line_len': min_line_len,
        'max_line_gap': max_line_gap,
        'thickness': None,
        'found_line': False
    }

    if lines is None or len(lines) == 0:
        return (pil_img, config)

    # Encontrar la línea más larga
    longest_len = 0
    longest_line = None
    for x1, y1, x2, y2 in lines[:, 0]:
        length = np.hypot(x2 - x1, y2 - y1)
        if length > longest_len:
            longest_len = length
            longest_line = (x1, y1, x2, y2)

    if longest_line is None:
        return (pil_img, config)

    # Borrar la línea más larga dibujando sobre ella en color blanco
    x1, y1, x2, y2 = longest_line
    thickness = random.randint(1, 3)
    config['thickness'] = thickness
    config['found_line'] = True

    # Dibujo en la imagen original (blanco = 255 en escala de grises)
    img_no_lines = img_cv.copy()
    cv2.line(img_no_lines, (x1, y1), (x2, y2), color=(255, 255, 255), thickness=thickness)

    return (Image.fromarray(img_no_lines), config)


# ------------------------------------------------------------
# 5) Función de OCR “fallback” con Tesseract + 30 variaciones + 5 ediciones de línea
# ------------------------------------------------------------

def preprocess_and_ocr_tesseract_with_edits(pil_img: Image.Image, cedula: str, intento: int) -> str:
    """
    1) Intenta 30 veces con Tesseract (variaciones adaptativeThreshold+morphology).
    2) Si aún no hay texto de 5 caracteres, realiza hasta 5 “ediciones de línea”:
       a) Borra la línea más larga con parámetros aleatorios y reintenta 30 pasadas Tesseract.
       b) Registra cada configuración de limpieza + Tesseract en el log.
    Devuelve la primera cadena válida de 5 caracteres, o "" si ninguna.
    """

    def tesseract_passes(img_gray: np.ndarray, cedula: str, intento: int, round_tag: str) -> str:
        """
        Ejecuta 30 pasadas de Tesseract con umbrales y kernels aleatorios.
        round_tag puede ser "BASE" o "EDIT{n}" para log.
        """
        elecciones_block = [11, 13, 15, 17, 19, 21]
        for idx in range(1, 31):  # 30 pasadas
            block_size = random.choice(elecciones_block)
            C = random.randint(1, 5)
            thresh = cv2.adaptiveThreshold(
                img_gray, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY_INV,
                block_size, C
            )
            k = random.choice([2, 3, 4])
            kernel = np.ones((k, k), np.uint8)

            if k >= 3:
                proc = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
                proc = cv2.morphologyEx(proc, cv2.MORPH_OPEN, kernel, iterations=1)
            else:
                proc = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

            proc_final = cv2.bitwise_not(proc)
            pil_for_tess = Image.fromarray(proc_final)

            config = {
                'round': round_tag,
                'tess_idx': idx,
                'block_size': block_size,
                'C': C,
                'kernel_size': k
            }
            txt = pytesseract.image_to_string(
                pil_for_tess,
                config=r"--psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
            )
            txt = txt.strip().replace(" ", "").replace("\n", "").replace("\r", "")
            valido = txt if len(txt) == 5 else ""

            # Registrar en log
            with open(RUTA_LOG, "a", encoding="utf-8") as logf:
                logf.write(
                    f"    {round_tag} TESSERA[{idx:2d}] | block={block_size:2d} "
                    f"C={C:1d} kernel={k:1d} | OCR='{valido}'\n"
                )

            if valido:
                return valido

        # Si ninguna pasada encontró 5 caracteres
        with open(RUTA_LOG, "a", encoding="utf-8") as logf:
            logf.write(f"    {round_tag} TESSERACT → NO_OCR_EN_30_VARIANTES\n\n")
        return ""

    # Convertir a escala de grises (OpenCV) una sola vez
    img_cv = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    gray_original = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    # 1) Intento base de 30 pasadas Tesseract
    with open(RUTA_LOG, "a", encoding="utf-8") as logf:
        logf.write(f"CÉDULA: {cedula} | TESSERACT_BASE_INTENTO: {intento}\n")

    texto = tesseract_passes(gray_original, cedula, intento, round_tag="BASE")
    if texto:
        return texto

    # 2) Si no hay texto, probar hasta 5 limpiezas de línea
    for ed in range(1, 6):
        # 2a) Limpiar la línea más larga
        with open(RUTA_LOG, "a", encoding="utf-8") as logf:
            logf.write(f"    --Limpieza {ed} para Cédula {cedula}, Intento {intento}\n")
        pil_edited, cfg_edit = remove_longest_line_random(pil_img, cedula, intento, ed)

        # Registrar configuración de edición
        with open(RUTA_LOG, "a", encoding="utf-8") as logf:
            logf.write(f"      Edit_cfg_{ed}: {cfg_edit}\n")

        # 2b) Convertir a gris y relanzar 30 pasadas de Tesseract
        img_edited_cv = cv2.cvtColor(np.array(pil_edited), cv2.COLOR_RGB2BGR)
        gray_edited = cv2.cvtColor(img_edited_cv, cv2.COLOR_BGR2GRAY)

        texto = tesseract_passes(gray_edited, cedula, intento, round_tag=f"EDIT{ed}")
        if texto:
            return texto

    # 3) Si tras 5 limpiezas sigue vacío, devolvemos ""
    return ""


# ------------------------------------------------------------
# 6) Función que abre el formulario inicial (igual que antes)
# ------------------------------------------------------------

def abrir_formulario(timeout: int = 15) -> webdriver.Chrome:
    chrome_opts = Options()
    chrome_opts.add_argument("--headless=new")
    chrome_opts.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    )
    chrome_opts.add_argument("--ignore-certificate-errors")
    chrome_opts.add_argument("--allow-insecure-localhost")
    chrome_opts.add_argument("--no-sandbox")
    chrome_opts.add_argument("--disable-dev-shm-usage")
    chrome_opts.add_argument("--incognito")
    chrome_opts.add_argument("--window-size=1920,1080")
    chrome_opts.add_argument("--log-level=3")

    driver = webdriver.Chrome(options=chrome_opts)

    try:
        url_inicio = "https://consultasrc.registraduria.gov.co:28080/ProyectoSCCRC/"
        driver.get(url_inicio)
        print("▶ Título inicial:", driver.title)
        time.sleep(1)

        boton = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "controlador:consultasId"))
        )
        boton.click()

        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.ID, "searchForm:tiposBusqueda"))
        )
        print("✔️ Llegaste al formulario de búsqueda.")
        return driver

    except Exception as e:
        driver.quit()
        raise RuntimeError(f"[abrir_formulario] Error: {e}")


# ------------------------------------------------------------
# 7) Función principal: buscar o generar certificado
# ------------------------------------------------------------

def buscar_o_generar_certificado(
    cedula: str,
    max_intentos: int = 3,
    fecha_nacimiento: str = ""
) -> str:
    """
    Para cada intento:
      1) Abre formulario en incógnito.
      2) Selecciona “DOCUMENTO (NUIP/NIP/Tarjeta de Identidad)” y rellena la cédula.
      3) Descarga la imagen CAPTCHA y la guarda como ORIGINAL_{cedula}_{intento}.png.
      4) Llama a resolver_captcha_con_easy_ensembles (20 variantes EasyOCR).
         - Si EasyOCR halla consenso, usamos ese texto.
         - Si falla, pasamos a preprocess_and_ocr_tesseract_with_edits
           (30 pasadas + 5 limpiezas de línea).
      5) Si obtenemos texto de 5 caracteres, lo ingresa y hace clic en “Buscar Registro Civil”.
      6) Verifica “No se han encontrado registros” (→ “NO_EXISTE”).
      7) Si aparece botón “Generar Certificado”, descarga PDF y devuelve “EXISTE”.
      8) Si tras 3 intentos no se resuelve CAPTCHA, devuelve “ERROR_CAPTCHA”.
      9) Otros fallos → “ERROR_GENERAL”.
    """
    for intento in range(1, max_intentos + 1):
        print(f"\n=== Intento {intento} de {max_intentos} para cédula {cedula} ===")
        driver = None
        try:
            # 1) Abrir el formulario
            driver = abrir_formulario(timeout=15)

            # 2) Seleccionar “DOCUMENTO (NUIP/NIP/Tarjeta de Identidad)”
            select_tipo = Select(driver.find_element(By.ID, "searchForm:tiposBusqueda"))
            select_tipo.select_by_visible_text("DOCUMENTO (NUIP/NIP/Tarjeta de Identidad)")

            WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, "searchForm:documento"))
            )

            input_ced = driver.find_element(By.ID, "searchForm:documento")
            input_ced.clear()
            input_ced.send_keys(cedula)

            # --------------------------------------------------------
            # 3) Descargar CAPTCHA original
            # --------------------------------------------------------
            img_elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//img[contains(@src,'kaptcha')]"))
            )
            captcha_src = img_elem.get_attribute("src")
            if captcha_src.startswith("data:image"):
                header, encoded = captcha_src.split(",", 1)
                img_bytes = base64.b64decode(encoded)
            else:
                url_base = "https://consultasrc.registraduria.gov.co:28080"
                full_url = requests.compat.urljoin(url_base, captcha_src)
                resp_img = requests.get(
                    full_url,
                    headers={"User-Agent": driver.execute_script("return navigator.userAgent;")}
                )
                img_bytes = resp_img.content

            pil_img = Image.open(io.BytesIO(img_bytes))

            nombre_captcha_original = f"ORIGINAL_{cedula}_{intento}.png"
            ruta_captcha_original = os.path.join(CARPETA_CAPTCHAS, nombre_captcha_original)
            with open(ruta_captcha_original, "wb") as f:
                f.write(img_bytes)
            print(f"▶ Guardé CAPTCHA original como: {ruta_captcha_original}")

            # --------------------------------------------------------
            # 4) Intentar resolver con EasyOCR ensemble (20 variantes)
            # --------------------------------------------------------
            texto_captcha, exito_easy = resolver_captcha_con_easy_ensembles(
                pil_img, cedula, intento
            )

            # --------------------------------------------------------
            # 5) Si EasyOCR falla, probamos con Tesseract + limpiezas de línea
            # --------------------------------------------------------
            if not exito_easy:
                print("⚠️ EasyOCR ensemble NO resolvió CAPTCHA, probamos con Tesseract + EDICIÓN…")
                texto_captcha = preprocess_and_ocr_tesseract_with_edits(pil_img, cedula, intento)
                if not texto_captcha:
                    print("❌ Tras Tesseract+ediciones no devolvió 5 caracteres. Ronda fallida.")
                    driver.quit()
                    continue
                else:
                    print(f"▶ Tesseract/Edición devolvió (fallback): '{texto_captcha}'")
            else:
                print(f"✔️ EasyOCR anunció CAPTCHA resuelto: '{texto_captcha}'")

            # --------------------------------------------------------
            # 6) Si llegamos aquí, texto_captcha tiene 5 caracteres
            # --------------------------------------------------------
            input_cap = driver.find_element(By.ID, "searchForm:inCaptcha")
            input_cap.clear()
            input_cap.send_keys(texto_captcha)

            if fecha_nacimiento:
                input_fecha = driver.find_element(By.ID, "searchForm:calendar1")
                input_fecha.clear()
                input_fecha.send_keys(fecha_nacimiento)

            boton_buscar = driver.find_element(By.ID, "searchForm:busquedaRCX")
            boton_buscar.click()
            time.sleep(2)

            # --------------------------------------------------------
            # 7) Verificar “No se han encontrado registros” o CAPTCHA incorrecto
            # --------------------------------------------------------
            try:
                div_error = WebDriverWait(driver, 3).until(
                    EC.visibility_of_element_located((By.ID, "searchForm:j_idt76"))
                )
                texto_error = div_error.text.strip()
                if "imagen de verificación" in texto_error:
                    print("❗ CAPTCHA INCORRECTO detectado (mensaje en pantalla).")
                    raise RuntimeError("CAPTCHA incorrecto")
                if "No se han encontrado registros" in texto_error:
                    print("❗ No se encontraron registros. (NO_EXISTE)")
                    return "NO_EXISTE"
            except Exception:
                # Si no aparece el div de error en 3s, seguimos
                pass

            # --------------------------------------------------------
            # 8) Buscar el botón “Generar Certificado”
            # --------------------------------------------------------
            try:
                boton_gen = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH,
                        "//input[@value='Generar Certificado']"
                    ))
                )
                print("✔️ Botón 'Generar Certificado' detectado. Descargando PDF…")
                boton_gen.click()
                time.sleep(2)

                handles = driver.window_handles
                if len(handles) > 1:
                    driver.switch_to.window(handles[-1])
                    time.sleep(1)

                url_pdf = driver.current_url
                print("▶ URL PDF:", url_pdf)

                headers = {"User-Agent": driver.execute_script("return navigator.userAgent;")}
                resp_pdf = requests.get(url_pdf, headers=headers)
                if resp_pdf.status_code == 200 and resp_pdf.content[:4] == b"%PDF":
                    nombre_pdf = f"certificado_{cedula}.pdf"
                    with open(nombre_pdf, "wb") as f:
                        f.write(resp_pdf.content)
                    print(f"✅ PDF guardado como '{nombre_pdf}'.")
                    return "EXISTE"
                else:
                    print("❌ ERROR: no se pudo descargar el PDF.")
                    return "ERROR_GENERAL"

            except Exception:
                print("⌛ Ni mensaje de error ni botón ‘Generar Certificado’ en tiempo. Reintentando…")
                driver.quit()
                continue

        except RuntimeError as err_int:
            print(f"⌛ Intento {intento} fallido: {err_int}")
            if "CAPTCHA incorrecto" in str(err_int):
                continue
            else:
                print("❌ Abortando: error inesperado.")
                return "ERROR_GENERAL"

        finally:
            if driver:
                driver.quit()

    # Si agotamos los intentos sin resolver el CAPTCHA
    print("⚠️ Tras varios intentos no se resolvió el CAPTCHA.")
    return "ERROR_CAPTCHA"


# ------------------------------------------------------------
# 8) Bloque principal (“main”) para probar con algunas cédulas
# ------------------------------------------------------------

if __name__ == "__main__":
    cedulas = [
        "1118573708",   # Ejemplo que NO EXISTE
        "1118540284",   # Ejemplo que SÍ EXISTE
        "1058352166"    # Ejemplo inválido
    ]

    for ced in cedulas:
        resultado = buscar_o_generar_certificado(
            cedula=ced,
            max_intentos=3,
            fecha_nacimiento=""  # Puedes poner “DD/MM/AAAA” si lo necesitas
        )
        print(f"\n► Resultado final para {ced}: {resultado}\n")

# data SET Registraduria

In [ ]:
import os
import time
import base64
import random
import requests
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    WebDriverException
)
from requests.exceptions import RequestException

# ------------------------------------------------------------
#  CONFIGURACIÓN
# ------------------------------------------------------------

# Directorio donde se guardarán los 5000 CAPTCHAs
DATASET_DIR = os.path.join(os.path.expanduser("~"), "Downloads", "Data_Set_Registraduria")
os.makedirs(DATASET_DIR, exist_ok=True)

# URL base de la Registraduría
BASE_URL = "https://consultasrc.registraduria.gov.co:28080/ProyectoSCCRC/"

# Número total de CAPTCHAs a descargar
TOTAL_CAPTCHAS = 5000

# Si interrumpes y quieres reanudar a partir de otro índice, cambia aquí:
START_INDEX = 1362  # <--- cambia este valor según necesites

# Rangos de espera (en segundos) entre descargas para evitar detección
MIN_SLEEP = 1.5
MAX_SLEEP = 3.0

# Tiempo de espera en caso de error de conexión antes de reintentar
NETWORK_BACKOFF = 10  # segundos

# ------------------------------------------------------------
#  FUNCIÓN PRINCIPAL
# ------------------------------------------------------------

def descargar_captchas():
    # 1) Configurar Chrome en modo headless + incógnito + user-agent “realista”
    chrome_opts = Options()
    chrome_opts.add_argument("--headless=new")
    chrome_opts.add_argument("--incognito")
    chrome_opts.add_argument("--disable-gpu")
    chrome_opts.add_argument("--no-sandbox")
    chrome_opts.add_argument("--disable-dev-shm-usage")
    chrome_opts.add_argument("--window-size=1920,1080")
    chrome_opts.add_argument("--ignore-certificate-errors")
    chrome_opts.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    )

    try:
        driver = webdriver.Chrome(options=chrome_opts)
    except WebDriverException as e:
        print(f"❌ Error iniciando ChromeDriver: {e}")
        return

    try:
        # Intentar cargar la URL inicial, con reintento si falla
        try:
            driver.get(BASE_URL)
        except WebDriverException:
            print("⚠️ No se pudo cargar la URL inicial. Reintentando en 10s...")
            time.sleep(NETWORK_BACKOFF)
            driver.get(BASE_URL)

        # Hacer clic en “Ingresar usuario público”
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "controlador:consultasId"))
            ).click()
        except (TimeoutException, WebDriverException):
            print("⚠️ No se encontró o no se pudo hacer clic en “Ingresar usuario público”. Reintentando en 10s...")
            time.sleep(NETWORK_BACKOFF)
            driver.get(BASE_URL)
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "controlador:consultasId"))
            ).click()

        # Esperar al dropdown de “Tipo de búsqueda”
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchForm:tiposBusqueda"))
        )

        # Preparar una sesión de requests que comparta cookies con Selenium
        session = requests.Session()
        for c in driver.get_cookies():
            session.cookies.set(c['name'], c['value'])

        # 5) Bucle para descargar cada CAPTCHA, desde START_INDEX hasta TOTAL_CAPTCHAS
        for i in range(START_INDEX, TOTAL_CAPTCHAS + 1):
            nombre_archivo = f"{i:05d}.png"
            ruta_archivo = os.path.join(DATASET_DIR, nombre_archivo)

            # Si ya existe ese archivo, saltar al siguiente
            if os.path.isfile(ruta_archivo):
                print(f"[{i:04d}/{TOTAL_CAPTCHAS:04d}] Ya existe {nombre_archivo}, saltando.")
                try:
                    img_elem = driver.find_element(By.XPATH, "//img[contains(@src,'kaptcha')]")
                    img_elem.click()
                    time.sleep(random.uniform(MIN_SLEEP, MAX_SLEEP))
                except Exception:
                    pass
                continue

            # A) Localizar el <img> del CAPTCHA
            try:
                img_elem = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//img[contains(@src,'kaptcha')]"))
                )
            except TimeoutException:
                print(f"⚠️ No se encontró la imagen del CAPTCHA en iteración {i}. Reintentando en 10s...")
                time.sleep(NETWORK_BACKOFF)
                driver.refresh()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "searchForm:tiposBusqueda"))
                )
                img_elem = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//img[contains(@src,'kaptcha')]"))
                )

            src_old = img_elem.get_attribute("src")

            # B) Descargar la imagen con hasta 3 reintentos
            for attempt in range(3):
                try:
                    if src_old.startswith("data:image"):
                        header, encoded = src_old.split(",", 1)
                        img_bytes = base64.b64decode(encoded)
                    else:
                        # Actualizar cookies antes de cada descarga
                        session.cookies.clear()
                        for c in driver.get_cookies():
                            session.cookies.set(c['name'], c['value'])
                        full_url = urljoin(BASE_URL, src_old)
                        resp = session.get(
                            full_url,
                            headers={"User-Agent": driver.execute_script("return navigator.userAgent;")},
                            timeout=15
                        )
                        resp.raise_for_status()
                        img_bytes = resp.content

                    # Guardar el archivo
                    with open(ruta_archivo, "wb") as f:
                        f.write(img_bytes)

                    print(f"[{i:04d}/{TOTAL_CAPTCHAS:04d}] CAPTCHA guardado: {ruta_archivo}")
                    break  # Salir del bucle de reintentos

                except (RequestException, IOError) as e:
                    print(f"⚠️ Error descargando CAPTCHA {i} (intento {attempt + 1}/3): {e}")
                    if attempt < 2:
                        time.sleep(NETWORK_BACKOFF)
                    else:
                        print(f"❌ No se pudo guardar CAPTCHA {i} tras 3 intentos, saltando.")
                        img_bytes = None
                        break

            # C) Si no se guardó, saltar al siguiente
            if not img_bytes:
                try:
                    img_elem.click()
                    time.sleep(random.uniform(MIN_SLEEP, MAX_SLEEP))
                except Exception:
                    driver.refresh()
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.ID, "searchForm:tiposBusqueda"))
                    )
                    time.sleep(random.uniform(MIN_SLEEP, MAX_SLEEP))
                continue

            # D) Generar un nuevo CAPTCHA haciendo click en la imagen
            try:
                img_elem.click()
            except Exception:
                # Si no funciona el click, recargamos la página completa
                driver.refresh()
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "controlador:consultasId"))
                ).click()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "searchForm:tiposBusqueda"))
                )

            # E) Esperar a que la nueva imagen sea diferente
            try:
                WebDriverWait(driver, 10).until(
                    lambda d: d.find_element(By.XPATH, "//img[contains(@src,'kaptcha')]").get_attribute("src") != src_old
                )
            except TimeoutException:
                time.sleep(2)

            # F) Esperar un intervalo aleatorio antes de continuar
            time.sleep(random.uniform(MIN_SLEEP, MAX_SLEEP))

    except Exception as e:
        print(f"❌ Error inesperado durante la descarga: {e}")
    finally:
        driver.quit()

    # 6) Verificar que existan las 5000 imágenes
    archivos = [f for f in os.listdir(DATASET_DIR) if f.lower().endswith(".png")]
    count = len(archivos)
    if count == TOTAL_CAPTCHAS:
        print(f"\n✅ Todas las {TOTAL_CAPTCHAS} imágenes se descargaron correctamente en:\n{DATASET_DIR}")
    else:
        print(f"\n⚠️ Se encontraron solo {count} imágenes en lugar de {TOTAL_CAPTCHAS}.")
        print("   Revisa la carpeta y vuelve a ejecutar o continúa desde el último número faltante.")


# ------------------------------------------------------------
#  EJECUCIÓN
# ------------------------------------------------------------

if __name__ == "__main__":
    descargar_captchas()
